# SQL 入門

> 資料分析

郭耀仁

In [1]:
# 連結資料庫
import sqlite3
import pandas as pd
from test_queries.test_queries_03 import extract_test_queries as etq

conn_nba = sqlite3.connect('nba.db')
conn_twelection = sqlite3.connect('twelection.db')

## 摘要

- 聚合與分組
- 進階的 SQL 查詢技巧
- 隨堂練習
- 隨堂練習參考解答

## 聚合與分組

## 我們可以將函式粗分為兩種類型

- 通用函式（Universal functions）
- 聚合函式（Aggregate functions）

## 其中通用函式的特性是輸入與輸出的列數相同

In [2]:
sql_query = """
SELECT firstName,
       UPPER(firstName) AS upper_first_name
  FROM players
 LIMIT 10;
"""

In [3]:
pd.read_sql(sql_query, conn_nba)

,firstName,upper_first_name
0,Vince,VINCE
1,Tyson,TYSON
2,LeBron,LEBRON
3,Carmelo,CARMELO
4,Kyle,KYLE
5,Udonis,UDONIS
6,Dwight,DWIGHT
7,Andre,ANDRE
8,JR,JR
9,Trevor,TREVOR


## 聚合函式的特性則是輸出的列數多半遠少於輸入的列數

In [4]:
sql_query = """
SELECT AVG(heightMeters) AS avg_height_meters
  FROM players;
"""
pd.read_sql(sql_query, conn_nba)

,avg_height_meters
0,1.99121


## SQLite 常用的聚合函式

<https://www.sqlitetutorial.net/sqlite-aggregate-functions/>

Source: <https://www.sqlitetutorial.net/>

## 使用 `COUNT()` 暸解表格的觀測值數

In [5]:
sql_query = """
SELECT COUNT(*) AS n_players
  FROM players;
"""
pd.read_sql(sql_query, conn_nba)

,n_players
0,504


## 使用 `COUNT()` 搭配 `PRAGMA_TABLE_INFO()` 暸解表格的欄位數

In [6]:
sql_query = """
SELECT COUNT(*) AS n_columns
  FROM PRAGMA_TABLE_INFO('players');
"""
pd.read_sql(sql_query, conn_nba)

,n_columns
0,20


## 使用 `MAX()` 與 `MIN()` 尋找最大與最小值

In [7]:
sql_query = """
SELECT MAX(heightMeters),
       MIN(heightMeters)
  FROM players;
"""
pd.read_sql(sql_query, conn_nba)

,MAX(heightMeters),MIN(heightMeters)
0,2.26,1.75


## 使用 `GROUP BY`

`GROUP BY` 如果單獨存在，作用與 `DISTINCT` 相同。

In [8]:
sql_query = """
SELECT confName,
       divName
  FROM teams
 GROUP BY divName
 ORDER BY confName, divName;
"""

In [9]:
pd.read_sql(sql_query, conn_nba)

,confName,divName
0,East,Atlantic
1,East,Central
2,East,Southeast
3,West,Northwest
4,West,Pacific
5,West,Southwest


## 使用 `GROUP BY` 搭配聚合函式就可以實現分組聚合

In [10]:
sql_query = """
SELECT country,
       COUNT(*) AS n_players
  FROM players
 GROUP BY country
 ORDER BY n_players DESC, country
 LIMIT 10;
"""

In [11]:
pd.read_sql(sql_query, conn_nba)

,country,n_players
0,USA,384
1,Canada,20
2,France,11
3,Australia,8
4,Croatia,6
5,Serbia,6
6,Germany,5
7,Latvia,4
8,Spain,4
9,Turkey,4


In [12]:
sql_query = """
SELECT pos,
       ROUND(AVG(heightMeters), 2) AS avgHeightMeters
  FROM players
 GROUP BY pos
 ORDER BY avgHeightMeters,
          pos;
"""
pd.read_sql(sql_query, conn_nba)

,pos,avgHeightMeters
0,G,1.90
1,G-F,1.98
2,F-G,2.00
3,F,2.03
4,F-C,2.09
5,C-F,2.10
6,C,2.12


## 使用 `HAVING` 篩選聚合後的數值

- `WHERE` 應對以單個「觀測值」為層級的數值
- `HAVING` 應對多個「觀測值」為層級的數值

In [13]:
sql_query = """
SELECT country,
       COUNT(*) AS n_players
  FROM players
 GROUP BY country
HAVING n_players > 5
 ORDER BY n_players DESC;
"""

In [14]:
pd.read_sql(sql_query, conn_nba)

,country,n_players
0,USA,384
1,Canada,20
2,France,11
3,Australia,8
4,Croatia,6
5,Serbia,6


In [15]:
sql_query = """
SELECT pos,
       ROUND(AVG(heightMeters), 2) AS avgHeightMeters
  FROM players
 GROUP BY pos
HAVING avgHeightMeters > 2
 ORDER BY avgHeightMeters, pos;
"""
pd.read_sql(sql_query, conn_nba)

,pos,avgHeightMeters
0,F,2.03
1,F-C,2.09
2,C-F,2.10
3,C,2.12


## 進階的 SQL 查詢技巧

## 使用 `CASE` 為資料重新歸類

使用條件敘述作重新歸類的依據。

```sql
CASE WHEN condition THEN result
     WHEN another_condition THEN result
     ELSE result
END AS variable_name
```

## `CASE` 的不同用法

- 將數值歸類成文字
- 將細項歸類成大項

In [16]:
sql_query = """
SELECT firstName,
       lastName,
       heightMeters,
       CASE WHEN heightMeters >= 2 THEN 'Taller than 2 m'
            ELSE 'Shorter than 2 m'
       END AS heightCategory
  FROM players
 LIMIT 10;
"""
pd.read_sql(sql_query, conn_nba)

,firstName,lastName,heightMeters,heightCategory
0,Vince,Carter,1.98,Shorter than 2 m
1,Tyson,Chandler,2.13,Taller than 2 m
2,LeBron,James,2.06,Taller than 2 m
3,Carmelo,Anthony,2.03,Taller than 2 m
4,Kyle,Korver,2.01,Taller than 2 m
5,Udonis,Haslem,2.03,Taller than 2 m
6,Dwight,Howard,2.08,Taller than 2 m
7,Andre,Iguodala,1.98,Shorter than 2 m
8,JR,Smith,1.98,Shorter than 2 m
9,Trevor,Ariza,2.03,Taller than 2 m


## 細項歸類成大項的根據

```python
{
    'G-F': 'G',
    'G': 'G',
    'F-G': 'F',
    'F-C': 'F',
    'F': 'F',
    'C-F': 'C',
    'C': 'C'
}
```

In [17]:
sql_query = """
SELECT firstName,
       lastName,
       pos,
       CASE WHEN pos IN ('G-F', 'G') THEN 'G'
            WHEN pos IN ('C-F', 'C') THEN 'C'
            ELSE 'F'
       END AS posNew
  FROM players
 LIMIT 10;
"""
pd.read_sql(sql_query, conn_nba)

,firstName,lastName,pos,posNew
0,Vince,Carter,G-F,G
1,Tyson,Chandler,C,C
2,LeBron,James,F,F
3,Carmelo,Anthony,F,F
4,Kyle,Korver,G-F,G
5,Udonis,Haslem,F,F
6,Dwight,Howard,C-F,C
7,Andre,Iguodala,G-F,G
8,JR,Smith,G,G
9,Trevor,Ariza,F,F


## 使用子查詢

子查詢指的是在一段查詢語法之中嵌入另一段需要先執行的查詢語法，將子查詢擺放至小括號之中並且加上縮排。

## 目前誰是生涯總得分最高的球員

- 首先要從 `careerSummaries` 著手
- 接著利用在前一個查詢獲得的 `MAX(points)` 對 `careerSummaries` 進行篩選
- 最後是利用前一個查詢獲得的 `personId` 對 `players` 進行篩選

In [18]:
sql_query = """
SELECT MAX(points)
  FROM careerSummaries;
"""
pd.read_sql(sql_query, conn_nba)

,MAX(points)
0,34087


In [19]:
sql_query = """
SELECT personId
  FROM careerSummaries
 WHERE points = 34087;
"""
pd.read_sql(sql_query, conn_nba)

,personId
0,2544


In [20]:
sql_query = """
SELECT firstName,
       lastName
  FROM players
 WHERE personId = 2544;
"""
pd.read_sql(sql_query, conn_nba)

,firstName,lastName
0,LeBron,James


In [21]:
sql_query = """
SELECT firstName,
       lastName
  FROM players
 WHERE personId = (
           SELECT personId
             FROM careerSummaries
            WHERE points = (
                SELECT MAX(points)
                  FROM careerSummaries
            )
       );
"""
pd.read_sql(sql_query, conn_nba)

,firstName,lastName
0,LeBron,James


## 目前湖人隊的球員陣容有誰

- 首先要從 `teams` 著手
- 接著利用在前一個查詢獲得的 `teamId` 對 `rosters` 進行篩選
- 最後是利用前一個查詢獲得的 `personId` 對 `players` 進行篩選

In [22]:
sql_query = """
SELECT teamId
    FROM teams
    WHERE nickname = 'Lakers';
"""
pd.read_sql(sql_query, conn_nba)

,teamId
0,1610612747


In [23]:
sql_query = """
SELECT personId
  FROM rosters
 WHERE teamId = 1610612747;
"""
pd.read_sql(sql_query, conn_nba)

,personId
0,2544
1,2730
2,2747
3,200765
4,201162
5,201580
6,201980
7,202340
8,202693
9,203076


In [24]:
sql_query = """
SELECT firstName,
       lastName
  FROM players
 WHERE personId IN (
           SELECT personId
             FROM rosters
            WHERE teamId = (SELECT teamId
                              FROM teams
                             WHERE nickname = 'Lakers')
       );
"""
pd.read_sql(sql_query, conn_nba)

,firstName,lastName
0,LeBron,James
1,Dwight,Howard
2,JR,Smith
3,Rajon,Rondo
4,Jared,Dudley
5,JaVale,McGee
6,Danny,Green
7,Avery,Bradley
8,Markieff,Morris
9,Anthony,Davis


## 使用子查詢來生成新欄位

依據鋒衛暸解 NBA 球員的組成：
- 首先要從 `players` 著手
- 接著利用在前一個查詢獲得的 `pos` 對 `players` 進行聚合
- 最後是利用前一個查詢獲得的 `n_players` 計算百分比

In [25]:
sql_query = """
SELECT COUNT(*)
  FROM players;
"""
pd.read_sql(sql_query, conn_nba)

,COUNT(*)
0,504


In [26]:
sql_query = """
SELECT pos,
       COUNT(*) player_count,
       504 AS ttl_player
  FROM players
 GROUP BY pos;
"""
pd.read_sql(sql_query, conn_nba)

,pos,player_count,ttl_player
0,C,33,504
1,C-F,28,504
2,F,137,504
3,F-C,39,504
4,F-G,26,504
5,G,176,504
6,G-F,65,504


In [27]:
sql_query = """
SELECT pos,
       COUNT(*) player_count,
       504 AS ttl_player,
       CAST(COUNT(*) AS REAL) / CAST(504 AS REAL) AS ratio
  FROM players
 GROUP BY pos;
"""
pd.read_sql(sql_query, conn_nba)

,pos,player_count,ttl_player,ratio
0,C,33,504,0.065476
1,C-F,28,504,0.055556
2,F,137,504,0.271825
3,F-C,39,504,0.077381
4,F-G,26,504,0.051587
5,G,176,504,0.349206
6,G-F,65,504,0.128968


In [28]:
sql_query = """
SELECT pos,
       CAST(COUNT(*) AS REAL) / (
           SELECT CAST(COUNT(*) AS REAL)
             FROM players
       ) AS ratio
  FROM players
 GROUP BY pos;
"""
pd.read_sql(sql_query, conn_nba)

,pos,ratio
0,C,0.065476
1,C-F,0.055556
2,F,0.271825
3,F-C,0.077381
4,F-G,0.051587
5,G,0.349206
6,G-F,0.128968


## 這是目前涵蓋的查詢保留字

使用 SQL 語法時，保留字順序必須要遵守。

```sql
SELECT DISTINCT CAST(column_name AS data_type) AS alias_name
  FROM table_name
 WHERE conditions
 GROUP BY column_name
HAVING conditions
 ORDER BY column_name DESC
 LIMIT n_obs;
```

## 隨堂練習

[資料分析：隨堂練習](https://mybinder.org/v2/gh/datainpoint/classroom-introduction-to-sql/master?filepath=03-exercises.ipynb)

## 隨堂練習：以 `COUNT()` 函式查詢 `teams` 暸解 NBA 東區（`confName = 'East'`）共有幾支球隊，將答案命名為 `n_teams`

In [29]:
expected_output = pd.read_sql(etq('0301'), conn_nba)

In [30]:
expected_output

,n_teams
0,15


## 隨堂練習：以 `COUNT()` 函式查詢 `teams` 暸解 NBA 西區（`confName = 'West'`）共有幾支球隊，將答案命名為 `n_teams`

In [31]:
expected_output = pd.read_sql(etq('0302'), conn_nba)

In [32]:
expected_output

,n_teams
0,15


## 隨堂練習：使用 `COUNT()` 函式查詢 `presidential2020` 有幾個觀測值，將答案命名為 `n_obs`

In [33]:
expected_output = pd.read_sql(etq('0303'), conn_twelection)

In [34]:
expected_output

,n_obs
0,51678


## 隨堂練習：使用 `COUNT()` 函式搭配 `PRAGMA_TABLE_INFO()` 函式查詢 `presidential2020` 有幾個欄位數，並命名為 `n_cols`

In [35]:
expected_output = pd.read_sql(etq('0304'), conn_twelection)

In [36]:
expected_output

,n_cols
0,7


## 隨堂練習：查詢 `careerSummaries` 生涯場均得分 `ppg` 超過 20 分的球員有幾位，並命名為 `n_players`

In [37]:
expected_output = pd.read_sql(etq('0305'), conn_nba)

In [38]:
expected_output

,n_players
0,18


## 隨堂練習：使用 `SUM()` 函式將 `presidential2020` 的票數加總，並命名為 `total_votes`

In [39]:
expected_output = pd.read_sql(etq('0306'), conn_twelection)

In [40]:
expected_output

,total_votes
0,14300940


## 隨堂練習：使用 `SUM()` 函式與 `GROUP BY` 將 `presidential2020` 三組候選人的票數加總，並命名為 `total_votes`，選擇 `number`、`candidates` 與 `total_votes` 這三個變數

In [41]:
expected_output = pd.read_sql(etq('0307'), conn_twelection)

In [42]:
expected_output

,number,candidates,total_votes
0,1,宋楚瑜/余湘,608590
1,2,韓國瑜/張善政,5522119
2,3,蔡英文/賴清德,8170231


## 隨堂練習：使用 `SUM()` 函式、 `GROUP BY` 與 `HAVING` 找出 `presidential2020` 臺北市總票數超過 10 萬的行政區，總票數命名為 `total_votes`，選擇 `town` 與 `total_votes` 這兩個變數

In [43]:
expected_output = pd.read_sql(etq('0308'), conn_twelection)

In [44]:
expected_output

,town,total_votes
0,大安區,182001
1,士林區,177249
2,內湖區,177062
3,文山區,167936
4,北投區,156354
5,中山區,141880
6,信義區,138516
7,松山區,125561
8,萬華區,118574


## 隨堂練習：使用 `CASE` 敘述，將 `presidential2020` 三組候選人的票數加總分類為六都（臺北市、新北市、桃園市、臺中市、臺南市與高雄市）和非六都，命名這個類別為 `county_type`，總票數命名為 `total_votes`，選擇 `county_type` 與 `total_votes` 這兩個變數

In [45]:
expected_output = pd.read_sql(etq('0309'), conn_twelection)

In [46]:
expected_output

,county_type,total_votes
0,六都,10039325
1,非六都,4261615


## 隨堂練習：使用 `CASE` 敘述，計算 `presidential2020` 三組候選人的在六都（臺北市、新北市、桃園市、臺中市、臺南市與高雄市）和非六都的得票數個為何，命名這個類別為 `county_type`，總票數命名為 `total_votes`，選擇 `number`、`candidates`、`county_type` 與 `total_votes` 這四個變數

In [47]:
expected_output = pd.read_sql(etq('0310'), conn_twelection)

In [48]:
expected_output

,number,candidates,county_type,total_votes
0,1,宋楚瑜/余湘,六都,427705
1,1,宋楚瑜/余湘,非六都,180885
2,2,韓國瑜/張善政,六都,3772174
3,2,韓國瑜/張善政,非六都,1749945
4,3,蔡英文/賴清德,六都,5839446
5,3,蔡英文/賴清德,非六都,2330785


## 隨堂練習：查詢 `nba.db` 目前生涯總助攻數（`assists`）最高的球員是誰

In [49]:
expected_output = pd.read_sql(etq('0311'), conn_nba)

In [50]:
expected_output

,firstName,lastName
0,Chris,Paul


## 隨堂練習：查詢 `nba.db` 目前生涯總籃板球 `totReb` 最高的球員是誰

In [51]:
expected_output = pd.read_sql(etq('0312'), conn_nba)

In [52]:
expected_output

,firstName,lastName
0,Dwight,Howard


## 隨堂練習：查詢 `nba.db` 目前公鹿隊（Milwaukee Bucks）的球員陣容有誰

In [53]:
expected_output = pd.read_sql(etq('0313'), conn_nba)

In [54]:
expected_output

,firstName,lastName
0,Kyle,Korver
1,Marvin,Williams
2,Ersan,Ilyasova
3,Brook,Lopez
4,Robin,Lopez
5,George,Hill
6,Wesley,Matthews
7,Eric,Bledsoe
8,Khris,Middleton
9,Giannis,Antetokounmpo


## 隨堂練習：查詢 `presidential2016` 三組候選人的得票率為何，選擇 `number`、`candidates` 與 `votes_percentage`

In [55]:
expected_output = pd.read_sql(etq('0314'), conn_twelection)

In [56]:
expected_output

,number,candidates,votes_percentage
0,1,朱立倫/王如玄,0.310409
1,2,蔡英文/陳建仁,0.561234
2,3,宋楚瑜/徐欣瑩,0.128357


## 隨堂練習：查詢 `presidential2020` 三組候選人的得票率為何，選擇 `number`、`candidates` 與 `votes_percentage`

In [57]:
expected_output = pd.read_sql(etq('0315'), conn_twelection)

In [58]:
expected_output

,number,candidates,votes_percentage
0,1,宋楚瑜/余湘,0.042556
1,2,韓國瑜/張善政,0.386137
2,3,蔡英文/賴清德,0.571307


## 隨堂練習

[資料分析：隨堂練習參考解答](https://mybinder.org/v2/gh/datainpoint/classroom-introduction-to-sql/master?filepath=03-suggested-answers.ipynb)